In [6]:
import os
new_directory = "C:/Users/shaur/Downloads"
os.chdir(new_directory)

print("Current working directory:", os.getcwd())

Current working directory: C:\Users\shaur\Downloads


In [3]:
!curl -L https://raw.githubusercontent.com/Denis2054/Transformers-for-NLP-2nd-Edition/master/Chapter04/kant.txt --output "kant.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  7 10.7M    7  841k    0     0  1600k      0  0:00:06 --:--:--  0:00:06 1603k
 27 10.7M   27 3008k    0     0  1980k      0  0:00:05  0:00:01  0:00:04 1980k
 46 10.7M   46 5119k    0     0  2030k      0  0:00:05  0:00:02  0:00:03 2030k
 62 10.7M   62 6888k    0     0  1958k      0  0:00:05  0:00:03  0:00:02 1958k
 78 10.7M   78 8587k    0     0  1900k      0  0:00:05  0:00:04  0:00:01 1900k
 93 10.7M   93 10.0M    0     0  1867k      0  0:00:05  0:00:05 --:--:-- 1895k
100 10.7M  100 10.7M    0     0  1867k      0  0:00:05  0:00:05 --:--:-- 1828k


In [7]:
%%time
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
paths=[str(x) for x in Path(".").glob("**/*.txt")]

tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>","<pad>","</s>","<unk>","<mask>",
])

CPU times: total: 4.39 s
Wall time: 1.81 s


In [8]:
import os
token_dir = 'C:/Users/shaur/Downloads/KantaiBERT'
if not os.path.exists(token_dir):
  os.makedirs(token_dir)
tokenizer.save_model('KantaiBERT')

['KantaiBERT\\vocab.json', 'KantaiBERT\\merges.txt']

In [9]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
tokenizer = ByteLevelBPETokenizer(
"./KantaiBERT/vocab.json",
"./KantaiBERT/merges.txt",
)

In [10]:
tokenizer.encode("The Critique of Pure Reason.").tokens

['The', 'ĠCritique', 'Ġof', 'ĠPure', 'ĠReason', '.']

In [11]:
tokenizer.encode("The Critique of Pure Reason.")

Encoding(num_tokens=6, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [12]:
tokenizer._tokenizer.post_processor = BertProcessing(
("</s>", tokenizer.token_to_id("</s>")),
("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [13]:
tokenizer.encode("The Critique of Pure Reason.").tokens

['<s>', 'The', 'ĠCritique', 'Ġof', 'ĠPure', 'ĠReason', '.', '</s>']

In [14]:
!nvidia-smi

Thu Aug 10 01:13:16 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.67                 Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   41C    P8              11W /  85W |      0MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [15]:
import torch
torch.cuda.is_available()

True

In [16]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [17]:
config

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

In [18]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("./KantaiBERT", max_length=512)

In [19]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)
print(model)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [20]:
print(model.num_parameters())

83504416


In [21]:
LP=list(model.parameters())
lp=len(LP)
print(lp)

106


In [22]:
np=0
for p in range(0,lp):
    PL2=True
    try:
        L2=len(LP[p][0]) #check if 2D
    except:
        L2=1 #not 2D but 1D
        PL2=False
    L1 = len(LP[p])
    L3 = L1*L2
    np+=L3
print(np)

83504416


In [23]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./kant.txt",
    block_size=128,
)

c:\Users\shaur\anaconda3\envs\denis\lib\site-packages\transformers\data\datasets\language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


CPU times: total: 3.8 s
Wall time: 12.8 s


In [24]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [25]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
output_dir="./KantaiBERT",
overwrite_output_dir=True,
num_train_epochs=1,
per_device_train_batch_size=64,
save_steps=10_000,
save_total_limit=2,
)
trainer = Trainer(
model=model,
args=training_args,
data_collator=data_collator,
train_dataset=dataset,
)

In [26]:
%%time
trainer.train()

c:\Users\shaur\anaconda3\envs\denis\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 19%|█▉        | 501/2672 [01:26<05:40,  6.37it/s]

{'loss': 6.6031, 'learning_rate': 4.06437125748503e-05, 'epoch': 0.19}


 37%|███▋      | 1001/2672 [02:47<04:39,  5.98it/s]

{'loss': 5.7308, 'learning_rate': 3.12874251497006e-05, 'epoch': 0.37}


 56%|█████▌    | 1501/2672 [04:09<03:20,  5.83it/s]

...

 94%|█████████▎| 2501/2672 [06:55<00:28,  5.99it/s]

{'loss': 4.8565, 'learning_rate': 3.218562874251497e-06, 'epoch': 0.94}


100%|██████████| 2672/2672 [07:24<00:00,  6.01it/s]

{'train_runtime': 444.4344, 'train_samples_per_second': 384.678, 'train_steps_per_second': 6.012, 'train_loss': 5.450710707795834, 'epoch': 1.0}
CPU times: total: 1min 59s
Wall time: 7min 24s


TrainOutput(global_step=2672, training_loss=5.450710707795834, metrics={'train_runtime': 444.4344, 'train_samples_per_second': 384.678, 'train_steps_per_second': 6.012, 'train_loss': 5.450710707795834, 'epoch': 1.0})

In [27]:
trainer.save_model("./KantaiBERT")

In [29]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./KantaiBERT",
    tokenizer="./KantaiBERT"
)

In [30]:
fill_mask("Human thinking involves human <mask>.")

[{'score': 0.04439457505941391,
  'token': 403,
  'token_str': ' reason',
  'sequence': 'Human thinking involves human reason.'},
 {'score': 0.014114857651293278,
  'token': 407,
  'token_str': ' object',
  'sequence': 'Human thinking involves human object.'},
 {'score': 0.01363371405750513,
  'token': 668,
  'token_str': ' principle',
  'sequence': 'Human thinking involves human principle.'},
 {'score': 0.012850186787545681,
  'token': 651,
  'token_str': ' understanding',
  'sequence': 'Human thinking involves human understanding.'},
 {'score': 0.011482062749564648,
  'token': 427,
  'token_str': ' conception',
  'sequence': 'Human thinking involves human conception.'}]